# Anyscale partner blog - Ray Data

## Prepare data

Use 48K IMDB dataset form Kaggle.

Import csv and save to parquet format.

In [1]:
# !python -m pip install pillow
# !python -m pip install tqdm
# !python -m pip install torch sentence-transformers langchain
# !python -m pip install pymilvus 'pymilvus[model]'

## Scratch - Data prep

In [2]:
# import pandas as pd
# import pprint, time, os

# df = pd.read_csv("data/final_data.csv")
# print(df.shape)
# df.head()


In [3]:
# # Concatenate 'Name', 'Keywords', and 'Description' into 'text' column
# df['text'] = df['Name'] + ' ' + df['Description'] + ' ' + df['ReviewBody']
# df['text'] = df['text'].fillna('')

# # Convert genres from string with commas in it to list of strings.
# df['Genres'] = df['Genres'].str.split(',')

# # Convert actors from string with commas in it to list of strings.
# df['Actors'] = df['Actors'].str.split(',')

# # Convert keywords from string with commas in it to list of strings.
# df['Keywords'] = df['Keywords'].str.split(',')

# # Extract out just the year from the date.
# def extract_year(movie_date):
#     try:
#         return int(movie_date.split('-')[0])
#     except Exception:
#         return -1  # return -1 instead of None
# df['MovieYear'] = df.DatePublished.apply(extract_year)

# # Drop extra columns.
# df.drop(columns=['RatingCount', 'BestRating', 'WorstRating'], inplace=True)
# df.drop(columns=['DatePublished'], inplace=True)
# df.drop(columns=['Description'], inplace=True)
# df.drop(columns=['ReviewBody'], inplace=True)

# # Inspect text.
# print(f"Example text length: {len(df.text[0])}")
# pprint.pprint(f"Example text: {df.text[0]}")

# print(df.dtypes)
# display(df.head(2))

In [4]:
# # Save top 100 rows as data/kaggle_imdb_small.csv.
# df.head(100).to_csv('data/kaggle_imdb_small.csv', index=False)

# # Save data as a parquet file.
# df.to_parquet('data/kaggle_imdb.parquet')
# df.head(100).to_parquet('data/kaggle_imdb_small.parquet', index=False)

In [5]:
# Install Ray.
# !pip install -U "ray[data,train,tune,serve,default]"

# Start Ray headnode local cluster.
# The command will print out the Ray cluster address, 
#    which can be passed to start the worker nodes.
# !ray start --head --port=6379

# For multi-node cluster, start Ray worker nodes.
# !ray start --address='http://127.0.0.1:8265'

# Submit a Ray job using local .py script.
# See https://docs.ray.io/en/latest/cluster/running-applications/job-submission/index.html 
# !export RAY_ADDRESS="http://127.0.0.1:8265"
# !ray job submit --working-dir . -- python ray_data_demo.py
  
#   See https://docs.ray.io/en/latest/cluster/running-applications/job-submission/index.html 
#   for more information on submitting Ray jobs to the Ray cluster.
  
#   To terminate the Ray runtime, run
#     ray stop
  
#   To view the status of the cluster, use
#     ray status
  
#   To monitor and debug Ray, view the dashboard at 
#     127.0.0.1:8265


# Read data into ray dataset.

In [6]:
import ray, pprint, time, os
from ray.data import read_parquet

# Start Ray before using ray functions.
ray.init(ignore_reinit_error=True)

# Example on doc page works.
# s3_uri = "s3://anonymous@air-example-data-2/imagenette2/train/"
# ds = ray.data.read_images(s3_uri, mode="RGB")

# Now try with local parquet file.
file_path = "data/kaggle_imdb_small.parquet"
ds = ray.data.read_parquet(file_path) # doesn't work

# Inspect the dataset.
print(f"Number rows: {ds.count()}")
# Display some metadata about the dataset.
print("\nMetadata: ")
print(ds)
# Take a peek at a single row
print("\nLook at a sample row:")
ds.take(1)

2024-03-28 20:56:03,165	INFO worker.py:1567 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-03-28 20:56:03,175	INFO worker.py:1743 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 


Parquet Files Sample 0:   0%|          | 0/1 [00:00<?, ?it/s]

2024-03-28 20:56:03,609	INFO dataset.py:2368 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2024-03-28 20:56:03,612	INFO streaming_executor.py:115 -- Starting execution of Dataset. Full log is in /tmp/ray/session_2024-03-28_16-56-29_410372_93015/logs/ray-data.log
2024-03-28 20:56:03,612	INFO streaming_executor.py:116 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> LimitOperator[limit=1]



Number rows: 100

Metadata: 
Dataset(
   num_rows=100,
   schema={
      id: int64,
      url: string,
      Name: string,
      PosterLink: string,
      Genres: list<element: string>,
      Actors: list<element: string>,
      Director: string,
      Keywords: list<element: string>,
      RatingValue: double,
      ReviewAurthor: string,
      ReviewDate: string,
      duration: string,
      text: string,
      MovieYear: int64
   }
)

Look at a sample row:


- ReadParquet->SplitBlocks(20) 1:   0%|          | 0/1 [00:00<?, ?it/s]

- limit=1 2:   0%|          | 0/1 [00:00<?, ?it/s]

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': 0,
  'url': 'https://www.imdb.com/title/tt2221420/',
  'Name': 'Sallie Gardner at a Gallop',
  'PosterLink': 'https://m.media-amazon.com/images/M/MV5BMjk0MzM1NDUtMTIyNi00Y2Y3LWEwNzEtMDQ1NmU4YzgzYWU2XkEyXkFqcGdeQXVyMjM3NTU0NTQ@._V1_.jpg',
  'Genres': ['Documentary', 'Short'],
  'Actors': ['Gilbert Domm', 'Sallie Gardner'],
  'Director': 'Eadweard Muybridge',
  'Keywords': ['19th century', '1870s', 'nature', 'horse', 'horse riding'],
  'RatingValue': 7.4,
  'ReviewAurthor': 'Cineanalyst',
  'ReviewDate': '2013-11-12',
  'duration': '1M',
  'text': 'Sallie Gardner at a Gallop Sallie Gardner at a Gallop is a short starring Gilbert Domm and Sallie Gardner. The clip shows a jockey, Domm, riding a horse, Sally Gardner. The clip is not filmed but instead consists of 24 individual photographs shot in rapid... Sometimes ascribed as "The Father of the Motion Picture", Eadweard Muybridge undeniably accomplished exploiting and sometimes introducing a means of instantaneous and serial images

# Load embedding model from Hugging Face.

In [7]:
import pymilvus
pymilvus.__version__

# Must be >= 2.4.0 in order for following code to work.

'2.4.0'

In [8]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

# Initialize a built-in Milvus sparse-dense-late-interaction-reranking encoder.
# https://huggingface.co/BAAI/bge-m3
encoder = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = encoder.dim["dense"]
print(f"dense_dim: {dense_dim}")

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

dense_dim: 1024


In [9]:
# # Quick test of the encoder.
# from typing import List

# df = pd.read_parquet("data/kaggle_imdb_small.parquet")
# print(df.shape)

# # Get the documents to embed
# docs = df['text'].to_list()

# # Ensure docs is a list of strings
# assert isinstance(docs, List)
# assert all(isinstance(doc, str) for doc in docs)

# # Encode the documents.
# docs_embeddings = encoder(docs)

# print(type(docs_embeddings['dense']), len(docs_embeddings['dense']), len(docs_embeddings['dense'][0]))

# Create a Milvus collection

In [10]:
# STEP 1. CONNECT TO ZILLIZ CLOUD

# !pip install pymilvus #python sdk for milvus
from pymilvus import connections, utility
import os
TOKEN = os.getenv("ZILLIZ_API_KEY")

# Connect to Zilliz cloud using endpoint URI and API key TOKEN.
# TODO change this.
CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
CLUSTER_ENDPOINT="https://in03-48a5b11fae525c9.api.gcp-us-west1.zillizcloud.com:443"
connections.connect(
  alias='default',
  #  Public endpoint obtained from Zilliz Cloud
  uri=CLUSTER_ENDPOINT,
  # API key or a colon-separated cluster username and password
  token=TOKEN,
)

# Check if the server is ready and get collection name.
print(f"Type of server: {utility.get_server_version()}")


Type of server: Zilliz Cloud Vector Database(Compatible with Milvus 2.3)


In [11]:
from pymilvus import MilvusClient

# Set the Milvus collection name.
COLLECTION_NAME = "imdb_metadata"

# Use no-schema Milvus client uses flexible json key:value format.
# https://milvus.io/docs/using_milvusclient.md
mc = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    # API key or a colon-separated cluster username and password
    token=TOKEN)

# Check if collection already exists, if so drop it.
has = utility.has_collection(COLLECTION_NAME)
if has:
    drop_result = utility.drop_collection(COLLECTION_NAME)
    print(f"Successfully dropped collection: `{COLLECTION_NAME}`")

# Create the collection using AUTOINDEX.
mc.create_collection(COLLECTION_NAME, 
                     dense_dim,
                     consistency_level="Eventually", 
                     auto_id=True,  
                     overwrite=True,
                    )

print(f"Successfully created collection: `{COLLECTION_NAME}`")
print(mc.describe_collection(COLLECTION_NAME))

Successfully dropped collection: `imdb_metadata`
Successfully created collection: `imdb_metadata`
{'collection_name': 'imdb_metadata', 'auto_id': True, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}], 'aliases': [], 'collection_id': 448076879578422052, 'consistency_level': 3, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True}


# Encode the text column

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import numpy as np
from typing import List

def recursive_splitter_wrapper(text, chunk_size):

    # Default chunk overlap is 10% chunk_size.
    chunk_overlap = np.round(chunk_size * 0.10, 0)

    # Use langchain's convenient recursive chunking method.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    chunks: List[str] = text_splitter.split_text(text)

    # Replace special characters with spaces.
    chunks = [text.replace("<br /><br />", " ") for text in chunks]

    return chunks

# Use recursive splitter to chunk text.
def imdb_chunk_text(df: pd.DataFrame) -> pd.DataFrame:
    BATCH_SIZE = 100
    CHUNK_SIZE = 512

    batch = df.head(BATCH_SIZE).copy()
    print(f"chunk size: {CHUNK_SIZE}")
    print(f"original shape: {batch.shape}")
    
    start_time = time.time()

    # 1. Chunk the text review into chunk_size.
    batch['chunk'] = batch['text'].apply(recursive_splitter_wrapper, chunk_size=CHUNK_SIZE)
    # Explode the 'chunk' column to create new rows for each chunk.
    batch = batch.explode('chunk', ignore_index=True)
    batch['chunk'] = batch['chunk'].fillna('')
    print(f"new shape: {batch.shape}")

    # 2. Add embeddings as new column in df.
    docs = batch['chunk'].to_list()
    # Ensure docs is a list of strings
    assert isinstance(docs, List)
    assert all(isinstance(doc, str) for doc in docs)
    # Encode the documents. bge-m3 dense embeddings will be already normalized.
    embeddings = encoder(docs)
    batch['vector'] = embeddings['dense']

    # 4. Drop the original 'text' column, keep the new 'chunk' column.
    batch.drop(columns=['text'], axis=1, inplace=True)

    end_time = time.time()
    print(f"Chunking + embedding time for {BATCH_SIZE} docs: {end_time - start_time} sec")
    assert len(batch.vector[0]) == dense_dim
    print(f"type embeddings: {type(batch.vector)} of {type(batch.vector[0])}")
    print(f"of numbers: {type(batch.vector[0][0])}")

    return batch

In [14]:
# # Quick test of the UDF function.
# from typing import List

# df = pd.read_parquet("data/kaggle_imdb_small.parquet")

# # Replace null values with empty strings.
# df['text'] = df['text'].fillna('')

# batch = imdb_chunk_text(df)
# display(batch.head(2))

In [18]:
# verify row count
print(f"Before transform: {ds.count()}")

# Transform the data using the UDF.
transformed_ds = ds.map_batches(imdb_chunk_text, batch_format="pandas")

# Inspect the dataset.
print(type(transformed_ds))
print(f"After transform number rows: {transformed_ds.count()}")
# print("\nMetadata: ")
# error printing vectors?
# # print(transformed_ds.materialize()) # doesn't work

print(transformed_ds) # shows the old ds, not the transformed_ds?

Before transform: 100
<class 'ray.data.dataset.Dataset'>
After transform number rows: 100
MapBatches(imdb_chunk_text)
+- Dataset(
      num_rows=100,
      schema={
         id: int64,
         url: string,
         Name: string,
         PosterLink: string,
         Genres: list<element: string>,
         Actors: list<element: string>,
         Director: string,
         Keywords: list<element: string>,
         RatingValue: double,
         ReviewAurthor: string,
         ReviewDate: string,
         duration: string,
         text: string,
         MovieYear: int64
      }
   )


# Shutdown ray

In [16]:
# if ray.is_initialized():
#     ray.shutdown()